# Kaggle House Prices Challenge

## House Prices: Advanced Regression Techniques

Predict sales prices and practice feature engineering, RFs, and gradient boosting

(Link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

First, importing the needed libraries for this project.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Loading the data into dataframes.

In [75]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [4]:
test.shape, train.shape

((1459, 80), (1460, 81))

In [5]:
test.columns, train.columns

(Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
        'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
        'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
        'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
        'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
        'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
        'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
        'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
        'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
        'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
        'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
        'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
        'GarageCond

We can see that the test dataset contains one more variable compared to the train dataset - which is the "SalePrice" variable. In our analysis / prediction this serves as the dependent variable we want to predict given the houses' characteristics.

In [6]:
train.SalePrice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

## Data preprocessing and cleaning

We'll start the Data Cleaning by checking if the dependent variable in the test dataset contains any missing values.

In [7]:
train.SalePrice.isnull().sum()

0

All observations contain data for the target variable, therefore we can continue by taking a look at all the other variables contained in the train and test dataset.

In [76]:
miss_count_train = train.isnull().sum().sort_values(ascending=False)
perc_miss_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missings_train = pd.concat([miss_count_train, perc_miss_train], axis=1, keys=["Total", "Percent"])

miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])

In [77]:
missings_train.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


In [78]:
missings_test.head(20)

,Total,Percent
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageCond,78,0.053461
GarageQual,78,0.053461
GarageYrBlt,78,0.053461
GarageFinish,78,0.053461


As a rule of thumb we completely ignore columns that contain at least 15% missing values and will not try to impute the missing values with any kind of computation, e.g. using means. Therefore, we will delete the variables "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu" and "LotFrontage".

In [70]:
train = train.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"])

The variables "GarageCond", "GarageType", "GarageQual", "GarageYrBlt" and "GarageFinish" contain exactly the same number of missing values, which seems kind of odd. Therefore, we'll take a closer look at these variables.

In [42]:
for var in ["GarageCond", "GarageType", "GarageQual", "GarageYrBlt", "GarageFinish"]:
    print(pd.crosstab(index=train[var], columns="count"))

col_0       count
GarageCond       
Ex              2
Fa             35
Gd              9
Po              7
TA           1326
col_0       count
GarageType       
2Types          6
Attchd        870
Basment        19
BuiltIn        88
CarPort         9
Detchd        387
col_0       count
GarageQual       
Ex              3
Fa             48
Gd             14
Po              3
TA           1311
col_0        count
GarageYrBlt       
1900.0           1
1906.0           1
1908.0           1
1910.0           3
1914.0           2
1915.0           2
1916.0           5
1918.0           2
1920.0          14
1921.0           3
1922.0           5
1923.0           3
1924.0           3
1925.0          10
1926.0           6
1927.0           1
1928.0           4
1929.0           2
1930.0           8
1931.0           4
1932.0           3
1933.0           1
1934.0           2
1935.0           4
1936.0           5
1937.0           2
1938.0           3
1939.0           9
1940.0          14
1941.0         

We can see that for "GarageCond" and "GarageQual" the most frequently occurring value is "TA", which means that the condition and quality of the garages are average/typical. We will replace the missing values of these two variables therefore with "TA" as well. The variable "GarageYrBlt" refers to the year in which the garage was built. Since we also have the year in which the houses themselves are built we can drop this variable without losing much explaining information. In addition to that we also drop the "GarageFinish" and "GarageType" variable.

In [71]:
train = train.drop(columns=["GarageYrBlt", "GarageFinish", "GarageType"])
train["GarageCond"] = train.GarageCond.fillna(value="TA")
train["GarageQual"] = train.GarageQual.fillna(value="TA")

In the same way as above we take a closer look at the "Bsmt*" variables.

In [65]:
for var in ["BsmtFinType2", "BsmtExposure", "BsmtCond", "BsmtFinType1", "BsmtQual"]:
    print(pd.crosstab(index=train[var], columns="count"))

col_0         count
BsmtFinType2       
ALQ              19
BLQ              33
GLQ              14
LwQ              46
Rec              54
Unf            1247
col_0         count
BsmtExposure       
Av              221
Gd              133
Mn              114
No              945
col_0     count
BsmtCond       
Fa           45
Gd           64
Po            2
TA         1303
col_0         count
BsmtFinType1       
ALQ             220
BLQ             148
GLQ             413
LwQ              74
Rec             132
Unf             427
col_0     count
BsmtQual       
Ex          120
Fa           35
Gd          611
TA          648


We delete the "BsmtFinType*" variables since these are highly subjective and do not add much information to our model. The missing values of "BsmtCond" will be imputed with the most common value "TA". The rows containing missing values for "BsmtQual" and "BsmtExposure" will be deleted from the dataset.

In [72]:
train = train.drop(columns=["BsmtFinType1", "BsmtFinType2"])
train.BsmtCond = train["BsmtCond"].fillna(value="TA")
for var in ["BsmtQual", "BsmtExposure"]:
    train = train.drop(train.loc[train[var].isnull()].index)

The variable "Electrical" contains only 1 missing value, therefore we only delete this specific row of data. We proceed in the same way with "MasVnrType" and "MasVnrArea".

In [73]:
for var in ["Electrical", "MasVnrType", "MasVnrArea"]:
    train = train.drop(train.loc[train[var].isnull()].index)

Running the above code again to check if all missing values are deleted.

In [74]:
miss_count_train = train.isnull().sum().sort_values(ascending=False)
perc_miss_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missings_train = pd.concat([miss_count_train, perc_miss_train], axis=1, keys=["Total", "Percent"])
missings_train

,Total,Percent
SalePrice,0,0.0
ExterQual,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
MasVnrArea,0,0.0
ExterCond,0,0.0
YearBuilt,0,0.0


Great! Since all missing observations are handled for the train data, we repeat the above steps for the test data.

In [82]:
test = test.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"])
test = test.drop(columns=["GarageYrBlt", "GarageFinish", "GarageType"])
test["GarageCond"] = test.GarageCond.fillna(value="TA")
test["GarageQual"] = test.GarageQual.fillna(value="TA")
test = test.drop(columns=["BsmtFinType1", "BsmtFinType2"])
test.BsmtCond = test["BsmtCond"].fillna(value="TA")
for var in ["BsmtQual", "BsmtExposure"]:
    test = test.drop(test.loc[test[var].isnull()].index)
for var in ["Electrical", "MasVnrType", "MasVnrArea"]:
    test = test.drop(test.loc[test[var].isnull()].index

In [84]:
miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])
missings_test

,Total,Percent
Exterior2nd,1,0.000716
KitchenQual,1,0.000716
MSZoning,1,0.000716
SaleType,1,0.000716
Functional,1,0.000716
Utilities,1,0.000716
GarageCars,1,0.000716
GarageArea,1,0.000716
Exterior1st,1,0.000716
SaleCondition,0,0.000000


We see that there are still missing observations for a few more variables, but that each of these contain only a single missing value. Therefore we drop the rows, for which there are missing values.

In [86]:
for var in ["Exterior2nd", "KitchenQual", "MSZoning", "SaleType", "Functional", "Utilities", "GarageCars", "GarageArea", "Exterior1st"]:
    test = test.drop(test.loc[test[var].isnull()].index)

In [87]:
miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])
missings_test

,Total,Percent
SaleCondition,0,0.0
MasVnrArea,0,0.0
YearRemodAdd,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
ExterQual,0,0.0
BsmtUnfSF,0,0.0


This looks perfect. We handled all missing data, deleted the columns, rows or did some imputation. Next, we continue with some feature engineering to enrich our dataset with some more features that can probably help explaining Sail Prices and finally can improve our model and predictions.

## Feature Engineering